# DrEvalPy demo
You can execute the DrEval Framework either via Nextflow as nf-core pipeline or as Python standalone.

In [24]:
!pip install drevalpy

First let us see which dataset and models are already implemented in drevalpy.
You can test your own model on all the datasets and comapre your model to all.the implemented ones:

In [25]:
from drevalpy.models import MODEL_FACTORY
from drevalpy.datasets import AVAILABLE_DATASETS
print(f"Models: {list(MODEL_FACTORY.keys())}")
print(f"Dataset: {list(AVAILABLE_DATASETS.keys())}")



Models: ['NaivePredictor', 'NaiveDrugMeanPredictor', 'NaiveCellLineMeanPredictor', 'NaiveMeanEffectsPredictor', 'NaiveTissueMeanPredictor', 'ElasticNet', 'RandomForest', 'SVR', 'SimpleNeuralNetwork', 'MultiOmicsNeuralNetwork', 'MultiOmicsRandomForest', 'GradientBoosting', 'SRMF', 'DIPK', 'ProteomicsRandomForest', 'ProteomicsElasticNet', 'SingleDrugRandomForest', 'MOLIR', 'SuperFELTR', 'SingleDrugElasticNet', 'SingleDrugProteomicsElasticNet', 'SingleDrugProteomicsRandomForest']
Dataset: ['GDSC1', 'GDSC2', 'CCLE', 'TOYv1', 'TOYv2', 'CTRPv1', 'CTRPv2']


In [26]:
# let us first train a model on the toy dataset. It will download the dataset for you.
from drevalpy.experiment import drug_response_experiment

naive_mean = MODEL_FACTORY["NaivePredictor"] # a naive model that just predicts the training mean
enet = MODEL_FACTORY["ElasticNet"] # An Elastic Net based on drug fingerprints and gene expression of 1000 landmark genes
simple_nn = MODEL_FACTORY["SimpleNeuralNetwork"] # A neural network based on drug fingerprints and gene expression of 1000 landmark genes

toyv2 = AVAILABLE_DATASETS["TOYv1"](path_data="data")

drug_response_experiment(
            models=[enet, simple_nn],
            baselines=[naive_mean], # Ablation studies and robustness tests are not done for baselines.
            response_data=toyv2,
            n_cv_splits=2, # the number of cross validation splits. Should be higher in practice :)
            test_mode="LCO", # LCO means Leave-Cell-Line out. This means that the test and validation splits only contain unseed cell lines.
            run_id="my_first_run",
            path_data="data", # where the downloaded drug response and feature data is stored
            path_out="results", # results are stored here :)
            hyperparameter_tuning=False) # if True (default), hyperparameters of the models and baselines are tuned.





Loading existing cv splits from results/my_first_run/TOYv1/LCO/splits
Running ElasticNet
- Full Test -
################# FOLD 1/2 #################
Split 0 already exists. Skipping.
################# FOLD 2/2 #################
Split 1 already exists. Skipping.
Running SimpleNeuralNetwork
- Full Test -
################# FOLD 1/2 #################
Split 0 already exists. Skipping.
################# FOLD 2/2 #################
Split 1 already exists. Skipping.
Running NaivePredictor
- Only Baseline Tests -
################# FOLD 1/2 #################
Split 0 already exists. Skipping.
################# FOLD 2/2 #################
Split 1 already exists. Skipping.
Done!


In [27]:
import os
import pandas as pd
os.listdir("results/my_first_run/TOYv1/LCO")
# the results folder holds splits and the results for all models. Lets look at the predictions of the simple neural network for the 0'th fold:
pd.read_csv("results/my_first_run/TOYv1/LCO/SimpleNeuralNetwork/predictions/predictions_split_0.csv")


,cell_line_name,pubchem_id,response,predictions,tissue
0,DU4475,123631,1.639112,2.105743,Breast
1,SK-MEL-31,123631,3.342029,2.782766,Skin
2,SK-MEL-24,637858,2.090905,2.687182,Skin
3,LNCaP clone FGC,11152667,0.359540,0.373855,Prostate
4,NCI-H2196,123631,2.912142,2.905978,Lung
...,...,...,...,...,...
866,HCC1143,3062316,0.306898,1.906021,Breast
867,Karpas-299,24821094,3.124550,0.458565,Lymph
868,Namalwa,24771867,-1.506492,-0.311268,Lymph
869,TE-10,36314,1.665760,-0.311268,Esophagus


In [28]:
# you can generate your own evaluations from these predictions.
# However, we recommend using our evaluation pipeline, which calculates meaningful metrics, creates figures and prepares an HTML report:
from drevalpy.visualization.create_report import create_report
create_report(run_id="my_first_run", dataset="TOYv1")

# this will create a report in the results/my_first_run/index.html which you can open in your browser.

Generating result tables ...
Evaluating file: "TOYv1/LCO/ElasticNet/predictions/predictions_split_0.csv" ...
Parsing file: /Users/piversen/Projects/munich/drevalpy/results/my_first_run/TOYv1/LCO/ElasticNet/predictions/predictions_split_0.csv
Calculating cell_line-wise evaluation measures …
Evaluating file: "TOYv1/LCO/ElasticNet/predictions/predictions_split_1.csv" ...
Parsing file: /Users/piversen/Projects/munich/drevalpy/results/my_first_run/TOYv1/LCO/ElasticNet/predictions/predictions_split_1.csv
Calculating cell_line-wise evaluation measures …
Evaluating file: "TOYv1/LCO/NaivePredictor/predictions/predictions_split_0.csv" ...
Parsing file: /Users/piversen/Projects/munich/drevalpy/results/my_first_run/TOYv1/LCO/NaivePredictor/predictions/predictions_split_0.csv
Calculating cell_line-wise evaluation measures …
Evaluating file: "TOYv1/LCO/NaivePredictor/predictions/predictions_split_1.csv" ...
Parsing file: /Users/piversen/Projects/munich/drevalpy/results/my_first_run/TOYv1/LCO/NaivePr

/Users/piversen/miniforge3/envs/drp2/lib/python3.13/site-packages/scikit_posthocs/_posthocs.py:735: RuntimeWarning:

divide by zero encountered in scalar divide



IndexError: list index out of range

In [ ]:
# We prefer running this in the console:

!drevalpy --models RandomForest --dataset_name TOYv1 --n_cv_splits 2 --test_mode LPO --run_id my_second_run --no_hyperparameter_tuning
!drevalpy-report --run_id my_second_run --dataset TOYv1

## Using the drevalpy nextflow pipeline for highly optimized runs:

You should use DrEval with Nextflow on high-performance clusters or clouds. Nextflow supports various systems like Slurm, AWS, Azure, Kubernetes, or SGE. On a local machine, you can also use the pipeline but probably, the overhang from spawning processes is not worth it so you might prefer the standalone. Nextflow needs a java version >=17, so we need to install that, too.

In [ ]:
!pip install nextflow
!apt-get install openjdk-17-jre-headless -qq > /dev/null
!java --version

In [ ]:
# we need a demo config for nextflow because on colab, we only have two CPUs available:
with open('demo.config', 'w') as f:
  f.write('process {\n')
  f.write('\tresourceLimits = [\n')
  f.write('\t\tcpus: 2,\n')
  f.write('\t\tmemory: "3.GB",\n')
  f.write('\t\ttime: "1.h",\n')
  f.write('\t]\n')
  f.write('}')

We run the pipeline with the TOYv1 dataset which was subset from CTRPv2. For the demo, we don't do hyperparameter tuning and we just do 2 CV splits. We want to inspect the final model which is why we train a final model on the full dataset. This should take about 10 minutes.
If you were on a compute cluster, you could now decide if you want to run the pipeline inside conda, docker, singularity, ... via the -profile option (-profile singularity, e.g.). If you want the executor to be slurm/..., you can write this in your config. You can find plenty of config examples online, e.g., the one for our group: [daisybio](https://github.com/nf-core/configs/blob/master/conf/daisybio.config)


In [ ]:
!nextflow run nf-core/drugresponseeval -r dev -c demo.config --dataset_name TOYv1 --models ElasticNet --baselines NaiveMeanEffectsPredictor --n_cv_splits 2 --no_hyperparameter_tuning --final_model_on_full_data

The results will be stored in `results/my_run`. You can inspect pipeline information like runtime or memory in `results/pipeline_info`. In `my_run/report`, you can find the html report where you can look at your results interactively. The underlying data is in `my_run/evaluation_results.csv` or `true_vs_pred.csv`.

We now inspect the final model saved in `results/my_run/LCO/ElasticNet/final_model` with `drevalpy` functions.

In [ ]:
from drevalpy.models import MODEL_FACTORY
enet_class = MODEL_FACTORY["ElasticNet"]
enet = enet_class.load("results/my_run/LCO/ElasticNet/final_model")
enet

We now want to extract the top scoring features.

In [ ]:
# get the top features
cell_line_input = enet.load_cell_line_features(data_path="data", dataset_name="TOYv1")
drug_input = enet.load_drug_features(data_path="data", dataset_name="TOYv1")
all_features = list(cell_line_input.meta_info['gene_expression'])+[f'fingerprint_{i}' for i in range(128)]

In [ ]:
import pandas as pd
df = pd.DataFrame({'feature': all_features, 'coef': enet.model.coef_})
df.sort_values(by="coef", ascending=False)

In [ ]:
print("Top 50 features:")
list(df.sort_values(by="coef", ascending=False)["feature"][:50])

The fingerprints are the most important features as the drug identity is responsible for the most variation between responses.

## Standalone demo

In [ ]:
from drevalpy.models import MODEL_FACTORY
from drevalpy.datasets import AVAILABLE_DATASETS
# First let us see which dataset and models are already implemented in drevalpy. You can test your own model on all the datasets and comapre your model to all.the implemented ones:
print(f"Models: {list(MODEL_FACTORY.keys())}")
print(f"Dataset: {list(AVAILABLE_DATASETS.keys())}")

# let us first train amodel on the toy dataset. It will download the dataset for you.
from drevalpy.experiment import drug_response_experiment

naive_mean = MODEL_FACTORY["NaivePredictor"]
rf = MODEL_FACTORY["ElasticNet"]
simple_nn = MODEL_FACTORY["SimpleNeuralNetwork"]

toyv2 = AVAILABLE_DATASETS["TOYv2"](path_data="data", measure="LN_IC50_curvecurator")

drug_response_experiment(
            models=[rf, simple_nn],
            baselines=[naive_mean],
            response_data=toyv2,
            metric="RMSE",
            n_cv_splits=2,
            test_mode="LCO",
            run_id="my_first_run",
            path_data="data",
            hyperparameter_tuning=False,
        )

In [ ]:
#run drevalpy in the console: